In [ ]:
pip install langchain together google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
%pip install --upgrade langchain-together

In [38]:
from langchain_together import ChatTogether
from langchain.schema import HumanMessage
import imaplib
import email
from email.header import decode_header
import smtplib
from email.mime.text import MIMEText
import json


In [74]:
api_key = 'f3b01991e5fcf24a0e024c67791f79a4f676aa55188df99925dafe784e82c9cf'
app_password = 'exnv kcle cnfy sjgf'
user_email = 'hnguyenphuong09@gmail.com'

llm = ChatTogether(
    together_api_key=api_key,
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"  # hoặc bất kỳ model chat nào bạn chọn
)


In [41]:
def read_emails(dummy:None):
    # Kết nối Gmail IMAP
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(user_email, app_password)
    mail.select("inbox")

    status, messages = mail.search(None, 'ALL')
    email_ids = messages[0].split()

    emails = []
    for eid in email_ids[-5:]:
        status, data = mail.fetch(eid, "(RFC822)")
        msg = email.message_from_bytes(data[0][1])
        subject, encoding = decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding or "utf-8", errors="ignore")

        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    body = part.get_payload(decode=True).decode(errors="ignore")
                    break
        else:
            body = msg.get_payload(decode=True).decode(errors="ignore")

        emails.append(f"📨 Tiêu đề: {subject}\n📩 Nội dung: {body[:200]}...")  # Giới hạn nội dung

    mail.logout()
    return "Đọc email thành công: Sau đây là danh sách email" +'\n\n---\n\n'.join(reversed(emails))


In [5]:
print(read_emails(None))  # Test thủ công trước khi nhúng vào agent

📨 Tiêu đề: Security alert
📩 Nội dung: [image: Google]
App password created to sign in to your account


hnguyenphuong09@gmail.com
If you didn't generate this password for AI_agent, someone might be using
your account. Check and secu...

---

📨 Tiêu đề: Security alert
📩 Nội dung: [image: Google]
App password used to sign in was removed


hnguyenphuong09@gmail.com
If you didn't remove this password, someone might be using your account.
Check and secure your account now.
...

---

📨 Tiêu đề: Security alert
📩 Nội dung: [image: Google]
App password used to sign in was removed


hnguyenphuong09@gmail.com
If you didn't remove this password, someone might be using your account.
Check and secure your account now.
...

---

📨 Tiêu đề: Security alert
📩 Nội dung: [image: Google]
App password used to sign in was removed


hnguyenphuong09@gmail.com
If you didn't remove this password, someone might be using your account.
Check and secure your account now.
...

---

📨 Tiêu đề: Security alert
📩 Nội dun

In [56]:
''.lstrip()

''

In [65]:
def send_email(input_string:str):
    data = json.loads(input_string)
    
    to = data.get('to') or data.get('recipient')
    to_list = [e.strip() for e in to.split(",")]  # Xử lý khoảng trắng
    subject = data.get('subject')
    body = data.get('body') or data.get('content')

    msg = MIMEText(body, "plain", "utf-8")
    msg["Subject"] = subject
    msg["From"] = user_email
    msg["To"] = ", ".join(to_list)  # Hiển thị danh sách người nhận

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(user_email, app_password)
        server.sendmail(user_email, to_list, msg.as_string())  # Gửi 1 lần cho tất cả

    print("✅ Đã gửi email qua Gmail.")

    return f"Đã gửi email email thành công tới người nhận {to} với nội dung {body}."

In [8]:
def create_calendar_event(summary: str, start_time: str, end_time: str):
    from googleapiclient.discovery import build
    from google.oauth2.credentials import Credentials

    creds = Credentials.from_authorized_user_file('token.json')
    service = build('calendar', 'v3', credentials=creds)

    event = {
        'summary': summary,
        'start': {'dateTime': start_time, 'timeZone': 'Asia/Ho_Chi_Minh'},
        'end': {'dateTime': end_time, 'timeZone': 'Asia/Ho_Chi_Minh'},
    }

    service.events().insert(calendarId='primary', body=event).execute()
    return "Sự kiện đã được tạo."


In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType

tools = [
    Tool(
        name="ReadEmails",
        func=read_emails,
        description="Dùng để đọc 5 email mới nhất từ hộp thư. Email đầu tiên là email mới nhất"
    ),
    Tool(
        name="SendEmail",
        func= send_email,
        description='Gửi email với chủ đề và nội dung cho một người nhận. Với định dạng {{ "to": "email1,email2", "subject": "subject", "body": "body" }} email1,email2 là danh sách email'
    ),
    Tool(
        name="CreateEvent",
        func=lambda x: create_calendar_event("Cuộc họp với team", "2025-07-08T10:00:00", "2025-07-08T11:00:00"),
        description="Tạo sự kiện lên lịch với tiêu đề, giờ bắt đầu và kết thúc."
    ),
    Tool(
        name="Search"
        func = 
        des = 'Dùng để check phòng nào còn trống'
    )
]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5
)

In [ ]:
message = 'hi'
summary='ngiuowiif dùng đã abc'
prompt = f'''
Bạn đóng vai trò là  1 trợ lý ảo
{summary}
{message}
'''

In [78]:
agent.run("gửi cho n21dccn069@student.ptithcm.edu.vn và n21dccn075@student.ptithcm.edu.vn nói rằng lựa lẹ con model nào đi không đuổi việc. trừ lương")



> Entering new AgentExecutor chain...
Để trả lời câu hỏi này, tôi cần gửi một email cho hai địa chỉ email đã cho với nội dung yêu cầu.

Action: SendEmail
Action Input: {"to": "n21dccn069@student.ptithcm.edu.vn,n21dccn075@student.ptithcm.edu.vn", "subject": "Lựa chọn model", "body": "Lựa lẹ con model nào đi không đuổi việc. Trừ lương"}✅ Đã gửi email qua Gmail.

Observation: Đã gửi email email thành công tới người nhận n21dccn069@student.ptithcm.edu.vn,n21dccn075@student.ptithcm.edu.vn với nội dung Lựa lẹ con model nào đi không đuổi việc. Trừ lương.
Thought:Tôi đã gửi email thành công tới người nhận với nội dung yêu cầu.

Thought: Tôi không cần thực hiện thêm bất kỳ hành động nào vì đã gửi email thành công.

Final Answer: Đã gửi email thành công tới người nhận n21dccn069@student.ptithcm.edu.vn,n21dccn075@student.ptithcm.edu.vn với nội dung Lựa lẹ con model nào đi không đuổi việc. Trừ lương.

> Finished chain.


'Đã gửi email thành công tới người nhận n21dccn069@student.ptithcm.edu.vn,n21dccn075@student.ptithcm.edu.vn với nội dung Lựa lẹ con model nào đi không đuổi việc. Trừ lương.'

web -> java -> đặt lịch ->
    <-       api call back

In [ ]:
api/v1/datphong
api/v1/huyphong
message
gọi lại đặt phòng
api tìm phòng trống
lưu Chat
web<-api callback <-agent

In [ ]:
Emmiter 